In [41]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.dates import DateFormatter
import os

# Data Prepping
- giving column names
- cutting to use only 5 best levels 


In [ ]:
dffff = pd.read_csv('data_message/INTC_2015-03-02_34200000_57600000_message_10.csv',index_col=None, header=0)
#lets set time using 
dffff['Time']  = pd.to_datetime(dffff['Time'], unit='s')


In [103]:
def load_message_book(date='2015-03-02'):
    # Load message file
    message_file = f'data_message/INTC_{date}_34200000_57600000_message_10.csv'
    
    # Load CSV with appropriate headers (assuming standard message book format)
    df = pd.read_csv(message_file, index_col=None, header=0)
    
    # Convert 'Time' from seconds since midnight to proper datetime
    df['Time'] = pd.to_timedelta(df['Time'], unit='s') + pd.to_datetime(date)
    
    return df

In [106]:
df_02 = load_message_book('2015-03-02')
df_03 = load_message_book('2015-03-03')
df_04 = load_message_book('2015-03-04')
df_05 = load_message_book('2015-03-05')
df_06 = load_message_book('2015-03-06')



# LOAD ORDERBOOK

In [116]:
import pandas as pd
import os

def loadOrderBook(theMessageBook, ticker='INTC', date='2015-03-02', data_dir='data'):
    """
    Load the order book for a given ticker and date, aligning it with message book timestamps.

    Args:
        theMessageBook (pd.DataFrame): Message book dataframe with timestamps.
        ticker (str): Stock ticker (e.g., 'INTC').
        date (str): Date in YYYY-MM-DD format.
        data_dir (str): Directory where order book files are stored.

    Returns:
        pd.DataFrame: Order book data indexed by the message book timestamps.
    """
    nlevels = 10  # Number of order book levels

    # Construct file path based on given structure
    order_book_file = os.path.join(data_dir, f"{ticker}_{date}_34200000_57600000_orderbook_10.csv")

    # Define column names dynamically
    col_types = ['Ask Price ', 'Ask Size ', 'Bid Price ', 'Bid Size ']
    columns = [f"{col}{level}" for level in range(1, nlevels + 1) for col in col_types]

    # Load the order book CSV
    order_book = pd.read_csv(order_book_file, names=columns)

    # Ensure message book contains a valid timestamp column
    if 'Time' not in theMessageBook.columns:
        raise ValueError("Message book must contain a 'Time' column for timestamps.")

    # Ensure the order book matches the message book length
    if len(order_book) != len(theMessageBook):
        raise ValueError(f"Mismatch: Message book has {len(theMessageBook)} rows, but order book has {len(order_book)}.")

    # Align order book index with message book timestamps
    order_book.index = theMessageBook['Time']

    # Convert price values from LOBSTER format (divide by 10,000)
    price_cols = [col for col in order_book.columns if 'Price' in col]
    order_book[price_cols] = order_book[price_cols] / 10000.0

    return order_book


In [119]:
df_ob_02 = loadOrderBook(df_02,date='2015-03-02')
df_ob_03 = loadOrderBook(df_03,date='2015-03-03')
df_ob_04 = loadOrderBook(df_04,date='2015-03-04')
df_ob_05 = loadOrderBook(df_05,date='2015-03-05')
df_ob_06 = loadOrderBook(df_06,date='2015-03-06')



In [120]:
#concat all dates to one dataframe
df_ob = pd.concat([df_ob_02, df_ob_03, df_ob_04, df_ob_05, df_ob_06])
df_ob

,Ask Price 1,Ask Size 1,Bid Price 1,Bid Size 1,Ask Price 2,Ask Size 2,Bid Price 2,Bid Size 2,Ask Price 3,Ask Size 3,...,Bid Price 8,Bid Size 8,Ask Price 9,Ask Size 9,Bid Price 9,Bid Size 9,Ask Price 10,Ask Size 10,Bid Price 10,Bid Size 10
Time,,,,,,,,,,,,,,,,,,,,,
2015-03-02 09:30:00.021332726,33.29,300,33.12,4300,33.40,50,33.07,300,33.45,300,...,32.75,1,33.64,100,32.50,12,34.50,40,32.43,100
2015-03-02 09:30:00.021332726,33.29,300,33.12,4300,33.32,200,33.07,300,33.40,50,...,32.75,1,33.60,50,32.50,12,33.64,100,32.43,100
2015-03-02 09:30:00.025798844,33.29,300,33.12,4300,33.32,400,33.07,300,33.40,50,...,32.75,1,33.60,50,32.50,12,33.64,100,32.43,100
2015-03-02 09:30:00.026162808,33.29,300,33.12,4300,33.31,200,33.07,300,33.32,400,...,32.75,1,33.57,100,32.50,12,33.60,50,32.43,100
2015-03-02 09:30:00.026249001,33.29,300,33.12,4300,33.31,200,33.07,300,33.32,400,...,32.75,1,33.57,100,32.50,12,33.60,50,32.43,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-03-06 15:59:59.920904705,33.20,145600,33.19,78322,33.21,100100,33.18,45701,33.22,95000,...,33.12,10434,33.28,23300,33.11,81730,33.29,5200,33.10,13208
2015-03-06 15:59:59.923091328,33.20,145600,33.19,78322,33.21,100100,33.18,45701,33.22,95000,...,33.12,10434,33.28,23300,33.11,81730,33.29,5200,33.10,13208
2015-03-06 15:59:59.923865688,33.20,145600,33.19,78200,33.21,100100,33.18,45701,33.22,95000,...,33.12,10434,33.28,23300,33.11,81730,33.29,5200,33.10,13208


In [121]:
#save to csv
df_ob.to_csv('order_book.csv', index=True)